# LlamaIndex Bottoms-Up Development - LLMs and Prompts
This notebook walks through testing an LLM using the primary prompt templates used in llama-index.

In [1]:
%%capture
!pip install llama-index
!pip install llama-index-llms-groq
!pip install llama-index-embeddings-huggingface
!pip install llama-parse

In [2]:
import os

os.environ["GROQ_API_KEY"] = "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"

In [3]:
from llama_index.llms.groq import Groq

llm = Groq(model="llama3-8b-8192")
llm_70b = Groq(model="llama3-70b-8192")

## Setup
In this section, we load a test document, create an LLM, and copy prompts from llama-index to test with.

First, let's load a quick document to test with. Right now, we will just load it as plain text, but we can do other operations later!

In [4]:
import requests

# URL of the file to download
url = "https://raw.githubusercontent.com/BoxOfCereal/llama_docs_bot_groq/refs/heads/main/docs/getting_started/starter_example.md"

# Download the file and save it locally
response = requests.get(url)
file_path = "starter_example.md"
with open(file_path, "wb") as file:
    file.write(response.content)

# Open and read the downloaded file
with open(file_path, "r") as f:
    text = f.read()

# print(text)  # This will print the content of the file


LlamaIndex uses some simple templates under the hood for answering queries -- mainly a `text_qa_template` for obtaining initial answers, and a `refine_template` for refining an existing answer when all the text does not fit into one LLM call.

Let's copy the default templates, and test out our LLM with a few questions.

In [5]:
from llama_index.core import Prompt

text_qa_template = Prompt(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

refine_template = Prompt(
    "We have the opportunity to refine the original answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question: {query_str}. "
    "If the context isn't useful, output the original answer again.\n"
    "Original Answer: {existing_answer}"
)

Now, lets test a few questions!

## Text QA Template Testing

In [6]:
question = "How can I install llama-index?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

According to the context information, you can install LlamaIndex by following the installation steps.


In [7]:
question = "How do I create an index?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

According to the context information, to create an index, you need to follow these steps:

1. Clone the LlamaIndex repository using the command `git clone https://github.com/jerryjliu/llama_index.git`
2. Navigate to the `examples` folder within the cloned repository using the command `cd llama_index`
3. Navigate to the `paul_graham_essay` folder using the command `cd examples/paul_graham_essay`
4. Create a new `.py` file and add the following code:
```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)
```
This code builds an index over the documents in the `data` folder.

Note that the `data` folder should contain the text files that you want to index. In this case, it's assumed to contain the text of Paul Graham's essay.


In [8]:
question = "How do I create an index? Write your answer using only code."
prompt = text_qa_template.format(context_str=text, query_str=question)
response_gen = llm.stream_complete(prompt)
for response in response_gen:
    print(response.delta, end="")

Here is the code snippet that creates an index:
```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)
```

## Refine Template Testing

In [9]:
question = "How do I create an index? Write your answer using only code."
existing_answer = """To create an index using LlamaIndex, you need to follow these steps:

1. Download the LlamaIndex repository by cloning it from GitHub.
2. Navigate to the `examples/paul_graham_essay` folder in the cloned repository.
3. Create a new Python file and import the necessary modules: `VectorStoreIndex` and `SimpleDirectoryReader`.
4. Load the documents from the `data` folder using `SimpleDirectoryReader('data').load_data()`.
5. Build the index using `VectorStoreIndex.from_documents(documents)`.
6. To persist the index to disk, use `index.storage_context.persist()`.
7. To reload the index from disk, use the `StorageContext` and `load_index_from_storage` functions.

Note: This answer assumes that you have already installed LlamaIndex and have the necessary dependencies."""
prompt = refine_template.format(context_msg=text, query_str=question, existing_answer=existing_answer)
response = llm.complete(prompt)
print(response.text)

Here is the refined answer:

```
from llama_index import VectorStoreIndex, SimpleDirectoryReader

# Load documents from the data folder
documents = SimpleDirectoryReader('data').load_data()

# Build the index
index = VectorStoreIndex.from_documents(documents)

# Persist the index to disk
index.storage_context.persist()
```


### Chat Example
The LLM also has a `chat` method that takes in a list of messages, to simulate a chat session.

In [16]:
from llama_index.core.base.llms.types import ChatMessage
# https://docs.llamaindex.ai/en/stable/api_reference/prompts/#llama_index.core.prompts.ChatMessage

chat_history = [
    ChatMessage(role="system", content="You are a helpful QA chatbot that can answer questions about llama-index."),
    ChatMessage(role="user", content="How do I create an index?"),
]

response = llm.chat(chat_history)
print(response.message)

assistant: Creating an index in Llama-Index is a straightforward process. Here's a step-by-step guide to help you get started:

1. **Install Llama-Index**: First, make sure you have Llama-Index installed on your machine. You can install it using pip: `pip install llama-index`.
2. **Create a new index**: Open a terminal or command prompt and navigate to the directory where you want to create your index. Run the following command to create a new index:
```
llma-index create <index_name>
```
Replace `<index_name>` with the name you want to give your index (e.g., "my_index").

3. **Configure the index**: After creating the index, you'll need to configure it. You can do this by running the following command:
```
llma-index config <index_name>
```
This will open the configuration file for your index in your default text editor. You can customize the settings to suit your needs.

4. **Add data to the index**: Once your index is configured, you can start adding data to it. You can do this by r

## Conclusion

In this notebook, we covered the low-level LLM API, and tested out some basic prompts with out documentation data.